In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
print(train.shape, test.shape)

(891, 12) (418, 11)


In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###과제 2번코드에서 작성한 데이터 변환을 이용해서 모델을 학습시키자
-'Age'도 4가지 범위로 분류되었으니 encoding을 하는데, 과제 1번에서 한 것을 바탕으로, 'Sex'는 순서가 없으니 one-hot encoding을, 'Embarked'와 'Age'는 Ordinal encoding을 하자.

-그외의 데이터 전처리 과정은 동일하게 진행한다.

In [82]:
age_tmp = train['Age'].copy()
age_tmp = pd.to_numeric(age_tmp, errors='coerce')
#replace any invalid numeric values (like the string labels we introduced) with NaN (Not a Number).

#'Age'col의 datatype를 float에서 문자열로 변경하기(에러메세지 뜸)
train['Age']=train['Age'].astype(str)

#train.loc[row_label, col_label]=value --> row_label, col_label에 해당하는 것을 value로 바꾸는 것
train.loc[age_tmp <= 17, 'Age'] = 'low'
train.loc[(age_tmp > 17) & (age_tmp <= 30), 'Age'] = 'mid1'
train.loc[(age_tmp > 30) & (age_tmp <= 50), 'Age'] = 'mid2'
train.loc[(age_tmp > 50), 'Age'] = 'high'
train

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  mid1      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  mid2      1   
2                               Heikkinen, Miss. Laina  female  mid1      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  mid2      1   
4                             Allen, Mr. William Henry    male  mid2      0   
..                                                 ...     ...   ...    ...   
886                              Montvila, Rev. Juozas    male  mid1      0   
887                       Graham, Miss. Margaret Edith  female  mid1      0   
888           Johnston, Miss. Catherine Helen "Carrie"  female   nan      1   
889                              Behr, Mr. Karl Howell    male  mid1      0   
890                                Dooley, Mr. Patrick    male  mid2      0   

     Parch            Ticket     Fare Cabin Embarked  
0        0         A/5 21171   7.2500   NaN        S  
1        0          PC 17599  71.2833   C85        C  
2        0  STON/O2. 3101282   7.9250   NaN        S  
3        0            113803  53.1000  C123        S  
4        0            373450   8.0500   NaN        S  
..     ...               ...      ...   ...      ...  
886      0            211536  13.0000   NaN        S  
887      0            112053  30.0000   B42        S  
888      2        W./C. 6607  23.4500   NaN        S  
889      0            111369  30.0000  C148        C  
890      0            370376   7.7500   NaN        Q  

[891 rows x 12 columns]

In [83]:
train = train.drop('Cabin', axis = 1)
train.Embarked = train.Embarked.fillna("C")

train['Age'] = train['Age'].map({'low':0, 'mid1':1, 'mid2':2, 'high':3})
train['Embarked'] = train['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

train = pd.get_dummies(train, columns=['Sex'])
train

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Age  SibSp  Parch  \
0                              Braund, Mr. Owen Harris  1.0      1      0   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  2.0      1      0   
2                               Heikkinen, Miss. Laina  1.0      0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  2.0      1      0   
4                             Allen, Mr. William Henry  2.0      0      0   
..                                                 ...  ...    ...    ...   
886                              Montvila, Rev. Juozas  1.0      0      0   
887                       Graham, Miss. Margaret Edith  1.0      0      0   
888           Johnston, Miss. Catherine Helen "Carrie"  NaN      1      2   
889                              Behr, Mr. Karl Howell  1.0      0      0   
890                                Dooley, Mr. Patrick  2.0      0      0   

               Ticket     Fare  Embarked  Sex_female  Sex_male  
0           A/5 21171   7.2500         2       False      True  
1            PC 17599  71.2833         0        True     False  
2    STON/O2. 3101282   7.9250         2        True     False  
3              113803  53.1000         2        True     False  
4              373450   8.0500         2       False      True  
..                ...      ...       ...         ...       ...  
886            211536  13.0000         2       False      True  
887            112053  30.0000         2        True     False  
888        W./C. 6607  23.4500         2        True     False  
889            111369  30.0000         0       False      True  
890            370376   7.7500         1       False      True  

[891 rows x 12 columns]

In [84]:
drop_cols = ['PassengerId', 'Name', 'Ticket']
train = train.drop(drop_cols, axis=1)

In [85]:
#data split
from sklearn.model_selection import train_test_split

# feature vector
X = train.drop('Survived', axis=1)
# target value
y = train['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(668, 8) (223, 8) (668,) (223,)


##3. RandomForestClassifier를 GridSearchCV 함수를 사용해서 hyper-parameter tuning을 수행하세요. hyper-parameter tuning을 수행할 파라미터는 max_depth, max_feature, n_estimators를 포함하여 5가지 이상 정해보세요. 어떤 값을 사용할지는 제한이 없습니다. 어떤 조합        에서 성능이 가장 잘 나왔는지 정리해보세요.

In [86]:
from sklearn.ensemble import RandomForestClassifier #만드는 모델
from sklearn.model_selection import GridSearchCV    #for 하이퍼파라미터 조정
from sklearn.metrics import accuracy_score          #for 만든 모델 평가

###하이퍼파라미터1

In [87]:
#파라미터1
clf = RandomForestClassifier()
params = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [6, 8, 10],
    'max_features': [0.5, 'sqrt']
}
grid_search = GridSearchCV(clf, params, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Parameters:  {'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 150}
Best Score:  0.8143754909662215


In [101]:
clf1 = RandomForestClassifier(**grid_search.best_params_)   #최적의 파라미터 넣어서 새로운 모델 생성
clf1.fit(X_train, y_train)

clf1_predict = clf1.predict(X_train)
clf1_val_predict = clf1.predict(X_val)

###하이퍼파라미터2


In [98]:
clf = RandomForestClassifier()
params2 = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [6, 8, 10],
    'max_features': [0.5, 'sqrt'],
    'min_samples_split': [2, 6, 10]
}
grid_search2 = GridSearchCV(clf, params2, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search2.fit(X_train, y_train)

print("Best Parameters: ", grid_search2.best_params_)
print("Best Score: ", grid_search2.best_score_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters:  {'max_depth': 6, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
Best Score:  0.820379306475143


In [102]:
clf2 = RandomForestClassifier(**grid_search2.best_params_)   #최적의 파라미터 넣어서 새로운 모델 생성
clf2.fit(X_train, y_train)

clf2_predict = clf2.predict(X_train)
clf2_val_predict = clf2.predict(X_val)

###하이퍼파라미터3

In [91]:
clf = RandomForestClassifier()
params3 = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [6, 8, 10],
    'max_features': [0.5, 'sqrt'],
    'min_samples_split': [2, 6, 10],
    'min_samples_leaf': [2, 6]}

grid_search3 = GridSearchCV(clf, params3, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search3.fit(X_train, y_train)

print("Best Parameters: ", grid_search3.best_params_)
print("Best Score: ", grid_search3.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best Parameters:  {'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
Best Score:  0.8233419369318821


In [103]:
clf3 = RandomForestClassifier(**grid_search3.best_params_)   #최적의 파라미터 넣어서 새로운 모델 생성
clf3.fit(X_train, y_train)

clf3_predict = clf3.predict(X_train)
clf3_val_predict = clf3.predict(X_val)

##하이퍼파라미터4

In [93]:
clf = RandomForestClassifier()
params4 = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [6, 8, 10],
    'max_features': [0.5, 'sqrt'],
    'min_samples_leaf': [2, 6]}

grid_search4 = GridSearchCV(clf, params4, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search4.fit(X_train, y_train)

print("Best Parameters: ", grid_search4.best_params_)
print("Best Score: ", grid_search4.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters:  {'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 200}
Best Score:  0.8173493435080237


In [104]:
clf4 = RandomForestClassifier(**grid_search4.best_params_)   #최적의 파라미터 넣어서 새로운 모델 생성
clf4.fit(X_train, y_train)

clf4_predict = clf4.predict(X_train)
clf4_val_predict = clf4.predict(X_val)

##하이퍼파라미터5

In [107]:
clf = RandomForestClassifier()
params5 = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [6, 8, 10],
    'max_features': [0.5, 'sqrt'],
    'min_samples_split': [2, 6, 10],
    'min_samples_leaf': [2, 6],
    'criterion': ['gini', 'entropy']}
grid_search5 = GridSearchCV(clf, params5, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search5.fit(X_train, y_train)

print("Best Parameters: ", grid_search5.best_params_)
print("Best Score: ", grid_search5.best_score_)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Best Parameters:  {'criterion': 'entropy', 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 100}
Best Score:  0.8263270115587475


In [108]:
clf5 = RandomForestClassifier(**grid_search5.best_params_)   #최적의 파라미터 넣어서 새로운 모델 생성
clf5.fit(X_train, y_train)

clf5_predict = clf5.predict(X_train)
clf5_val_predict = clf5.predict(X_val)

##하이퍼파리미터를 다르게 조정하여 학습시킨 모델 5개를 평가

In [111]:
print("clf1 ACC :")
print("Train ACC : %.4f" %accuracy_score(y_train, clf1_predict))
print("Validation ACC : %.4f" %accuracy_score(y_val, clf1_val_predict))
print()
print("clf2 ACC :")
print("Train ACC : %.4f" %accuracy_score(y_train, clf2_predict))
print("Validation ACC : %.4f" %accuracy_score(y_val, clf2_val_predict))
print()
print("clf3 ACC :")
print("Train ACC : %.4f" %accuracy_score(y_train, clf3_predict))
print("Validation ACC : %.4f" %accuracy_score(y_val, clf3_val_predict))
print()
print("clf4 ACC :")
print("Train ACC : %.4f" %accuracy_score(y_train, clf4_predict))
print("Validation ACC : %.4f" %accuracy_score(y_val, clf4_val_predict))
print()
print("clf5 ACC :")
print("Train ACC : %.4f" %accuracy_score(y_train, clf5_predict))
print("Validation ACC : %.4f" %accuracy_score(y_val, clf5_val_predict))

clf1 ACC :
Train ACC : 0.8728
Validation ACC : 0.8027

clf2 ACC :
Train ACC : 0.8757
Validation ACC : 0.8161

clf3 ACC :
Train ACC : 0.8518
Validation ACC : 0.8251

clf4 ACC :
Train ACC : 0.8548
Validation ACC : 0.8161

clf5 ACC :
Train ACC : 0.8623
Validation ACC : 0.8161
